# Calculation

Let look at one period of $R$ days.  What are the chances of losing
data in that period?  For that to happen, P+1 shards (or more) would have to fail 
in that period.

Over one year, the chances that a shard will fail is evenly distributed over
all of the $R$-day periods in the year.  We will use $F$ to denote the failure
rate of one shard in an $R$-day period:

> $F = A\frac{R}{365}$

To precisely compute the probability of failure during one period, you need to
translate the failure rate into a failure probability.  Usually, we use a [Poisson
distribution](https://en.wikipedia.org/wiki/Poisson_distribution) for failures to
compute the probability.  Given a failure rate $F$, the probability of one or more
failures is $1 - e^F$.  For small $F$, this is very close to $F$, which is what we'll
use here for simplicity.  So we will be a little sloppy and use $F$ both for the 
failure rate, and for the probability of failure.

Given the probability of one shard failing, the probability of $n$ specific shards
failing is obtained by multiplying their probablities together, because we have 
assumed that their failures are independent:

> $F^n$
    
That was the probability for *n* specific shards.  What we care about is the
probability of losing any *n* shards.  For that we multiply the probability above
times the number of ways to choose *n* shards from the full set of D+P shards:
 
> $\binom{n}{D + P} F^n$
    
We also lose data if more than n shards fail in the period.  To include those,
we can sum the above formula for n through D+P shards:

> $\sum_{i=n}^{D+P} F^i$
    
The durability in each period is inverse of that.  Durability over the full year 
happens when there's durability in all of the periods, which is the product of
probabilities:

> $\prod_{1}^{365/R} (1 - \sum_{i=n}^{D+P} F^i)$
